# Problem 194: Coloured Configurations

Consider graphs built with the units $A$: 

![](./0194_GraphA.png)

and $B$: 

![](./0194_GraphB.png)

where the units are glued along the vertical edges as in the graph 

![](./0194_Fig.png)

A configuration of type $(a, b, c)$ is a graph thus built of $a$ units $A$ and $b$ units $B$, where the graph's vertices are coloured using up to $c$ colours, so that no two adjacent vertices have the same colour.
The compound graph above is an example of a configuration of type $(2,2,6)$, in fact of type $(2,2,c)$ for all $c \ge 4$.

Let $N(a, b, c)$ be the number of configurations of type $(a, b, c)$.
For example, $N(1,0,3) = 24$, $N(0,2,4) = 92928$ and $N(2,2,3) = 20736$.

Find the last $8$ digits of $N(25,75,1984)$.

I label each vertex from 0 to 6 starting from the bottom left. Then I define graph $A$ and $B$ as a dictionary where each key is the vertex and the value is the vertices of its neighbours.

For sanity checking I also include he complete graph condisting of three vertices that are all connected. Such a graph has a chromatic polynomial of

$$P(G_\mathrm{complete}, k) = k \times (k-1) \times (k-2)$$

In [65]:
from dataclasses import dataclass
from collections import defaultdict


@dataclass
class Graph:
    definition: dict[list[int]]

    def concat(self, other: "Graph") -> "Graph":
        newGraph = defaultdict(list[int])
        offset = len(self.definition) - 2
        for leftKey, leftValues in self.definition.items():
            newGraph[leftKey].extend(leftValues)
        for rightKey, rightValues in other.definition.items():
            newGraph[rightKey + offset].extend([v + offset for v in rightValues])
        for k, v in newGraph.items():
            newGraph[k] = sorted(list(set(newGraph[k])))

        return Graph(newGraph)


graphA = Graph(
    {
        0: [1, 2, 5],
        1: [0, 4, 6],
        2: [0, 3, 5],
        3: [2, 4],
        4: [1, 3, 6],
        5: [0, 2, 6],
        6: [1, 4, 5],
    }
)

graphB = Graph(
    {
        0: [1, 2, 5],
        1: [0, 4],
        2: [0, 3, 5],
        3: [2, 4],
        4: [1, 3, 6],
        5: [0, 2, 6],
        6: [4, 5],
    }
)

completeGraph = Graph({0: [1, 2], 1: [0, 2], 2: [0, 1]})

I create a backtracking algorithm that can calculate the number of coloured configurations for small graphs

In [66]:
def countColouredConfigurations(input: "Graph", numColors=int) -> int:
    if numColors < 1:
        return 0
    colors = [x for x in range(numColors)]

    graph = input.definition

    def backtrack(count: int, vertex: int, currentColors: list[int]) -> int:
        if vertex == 0:  # first pass
            currentColors[0] = colors[0]
            return backtrack(count, 1, currentColors.copy())
        elif vertex >= len(graph):  # bottom of spanning tree
            if currentColors[-1] >= 0:
                return 1
            else:
                return 0
        else:
            if currentColors[vertex - 1] < 0:  # no valid colorings
                return 0
            counts = []
            for color in colors:
                if color not in [
                    currentColors[neighbour] for neighbour in graph[vertex]
                ]:
                    currentColors[vertex] = color
                    counts.append(backtrack(count, vertex + 1, currentColors.copy()))
            return sum(counts)

    currentColors = [-1 for x in range(len(graph))]
    cnt = backtrack(0, 0, currentColors)
    return numColors * cnt


# chromatic polynomial of complete graph with 3 vertices
def PcompleteGraph(x: int) -> int:
    return x * (x - 1) * (x - 2)


print("Backtracking algoritm")
print([countColouredConfigurations(completeGraph, n) for n in range(1, 11)])
print("")
print("chromatic polynomial")
print([PcompleteGraph(n) for n in range(1, 11)])

Backtracking algoritm
[0, 0, 6, 24, 60, 120, 210, 336, 504, 720]

chromatic polynomial
[0, 0, 6, 24, 60, 120, 210, 336, 504, 720]


If I give the above sequence to Wolfram Alpha it is able to infer the correct chromatic polynomial, so the idea is to try and do the same for graphs $A$ and $B$

In [67]:
print("Sequence: Graph A")
print([countColouredConfigurations(graphA, n) for n in range(1, 11)])
print("")
print("Sequence: Graph B")
print([countColouredConfigurations(graphB, n) for n in range(1, 11)])

Sequence: Graph A
[0, 0, 24, 744, 7440, 41880, 167160, 530544, 1429344, 3404880]

Sequence: Graph B
[0, 0, 36, 1056, 9720, 51840, 199500, 616896, 1629936, 3824640]


In [68]:
def PGraphA(n: int) -> int:
    return (n - 2) * (n - 1) * (n**5 - 7 * n**4 + 20 * n**3 - 29 * n**2 + 19 * n)


def PGraphB(n: int) -> int:
    return (n - 2) * (n - 1) * (n**5 - 6 * n**4 + 15 * n**3 - 20 * n**2 + 12 * n)

In [69]:
# Sanity checks

print("Graph A 15 colours")
print("Backtracking algorithm:", countColouredConfigurations(graphA, 15))
print("Chromatic polynomial:", PGraphA(15))
print("")
print("Graph B 15 colours")
print("Backtracking algorithm:", countColouredConfigurations(graphB, 15))
print("Chromatic polynomial:", PGraphB(15))

Graph A 15 colours
Backtracking algorithm: 84859320
Chromatic polynomial: 84859320

Graph B 15 colours
Backtracking algorithm: 91351260
Chromatic polynomial: 91351260


To calculate the number possible coloured configurations, we note that the chromatic polynomial of two graphs summed on a clique consisting of n vertices is

$$\frac{P(G_1, k) \times P(G_2, k)}{P(G_\mathrm{n-clique}, k)}$$

Since the configurations in the problem are joined on a two clique which has chromatic polynomial

$$P(G_\mathrm{2-clique}, k) = k \times (k - 1)$$

it's easy to calculate the number of coloured configurations

In [113]:
from functools import reduce
from math import factorial


def PTwoClique(n: int) -> int:
    return n * (n - 1)


def N(a: int, b: int, ncolors: int, modolu=10**8) -> int:
    twoClique = PTwoClique(ncolors)
    multinomialCoefficient = factorial(a + b) // (factorial(a) * factorial(b))
    individualConfigurations = (
        (
            pow(PGraphA(ncolors) // twoClique, a)
            * pow(PGraphB(ncolors) // twoClique, b)
            % modolu
        )
        * twoClique
        % modolu
    ) % modolu
    return multinomialCoefficient % modolu * individualConfigurations % modolu % modolu


print(N(1, 0, 3))
print(N(0, 2, 4))
print(N(2, 2, 3))
print(N(25, 75, 1984))

24
92928
20736
61190912


In [86]:
graphAA = graphA.concat(graphA)
graphBB = graphB.concat(graphB)
graphAB = graphA.concat(graphB)
graphBA = graphB.concat(graphA)
graphAAA = graphA.concat(graphA).concat(graphA)
graphBAA = graphB.concat(graphA).concat(graphA)
graphABA = graphA.concat(graphB).concat(graphA)
graphAAB = graphA.concat(graphA).concat(graphB)
graphAAAA = graphA.concat(graphA).concat(graphA).concat(graphA)
graphABBA = graphA.concat(graphB).concat(graphB).concat(graphA)

In [87]:
print("Sequence: Graph A")
print([countColouredConfigurations(graphA, n) for n in range(1, 11)])
print("")
print("Sequence: Graph B")
print([countColouredConfigurations(graphB, n) for n in range(1, 11)])
print("")
print("Sequence: Graph AAAA")
print([countColouredConfigurations(graphAAAA, n) for n in range(1, 4)])
print("")
print("Sequence: Graph ABBA")
print([countColouredConfigurations(graphABBA, n) for n in range(1, 4)])
print("")
# print("Sequence: Graph AA")
# print([countColouredConfigurations(graphAA, n) for n in range(1, 5)])
# print("")
# print("Sequence: Graph BB")
# print([countColouredConfigurations(graphBB, n) for n in range(1, 5)])
# print("")
# print("Sequence: Graph AB")
# print([countColouredConfigurations(graphAB, n) for n in range(1, 5)])
# print("")
# print("Sequence: Graph BA")
# print([countColouredConfigurations(graphBA, n) for n in range(1, 5)])
# print("")
# print("Sequence: Graph AAA")
# print([countColouredConfigurations(graphAAA, n) for n in range(1, 5)])
# print("")
# print("Sequence: Graph BAA")
# print([countColouredConfigurations(graphBAA, n) for n in range(1, 5)])
# print("")
# print("Sequence: Graph ABA")
# print([countColouredConfigurations(graphABA, n) for n in range(1, 5)])
# print("")
# print("Sequence: Graph AAB")
# print([countColouredConfigurations(graphAAB, n) for n in range(1, 5)])
# print("")

Sequence: Graph A
[0, 0, 24, 744, 7440, 41880, 167160, 530544, 1429344, 3404880]

Sequence: Graph B
[0, 0, 36, 1056, 9720, 51840, 199500, 616896, 1629936, 3824640]

Sequence: Graph AAAA
[0, 0, 1536]

Sequence: Graph ABBA
[0, 0, 3456]

